In [1]:
# Things to do 
# Follow  hybrid approach for the model by creating 2 cosine similarity matrices, one which has the 
# similarity values for the individual books and another which has the similarity values for the individual users. 

# # It would be a good idea to create separate functions for the number of inputs that the user enters 
# # For upto 3 inputs the cosine similairty matrix of the books should be used, and the books that have 
# the highest cosine simialrity values to the books that were input should be recommended. 

# For 4 and 5 inputs you can try to profile the user to another user who has rated the same books very highly 
# and then recommend the most highly rated books of that user. There can be some issues in this approach though. 
# Firstly there can be multiple users have highly rated the books that were given as input so then how do we figure 
# out which user's profile is the most similar to input profile, do we just pick the user who has rated
#  the max no. of books?   
# #  And this user would have also probably rated a lot of books highly(more than 5 in number) but since we are only 
# providing 5 recs how do we decide which 5 to recommend?  



In [2]:
from google.colab import drive 
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
import os 
import numpy as np
import pandas as pd
import zipfile
import tensorflow as tf
from tensorflow import keras
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
os.getcwd()

'/content'

In [4]:
zf = zipfile.ZipFile('/content/gdrive/MyDrive/BookRec.zip') 

df = pd.read_csv(zf.open('book1300k-1400k.csv'))
# df.shape
df.head()
#df_copy=df.copy()
#df_copy.shape

,Id,Name,Authors,ISBN,Rating,PublishYear,PublishMonth,PublishDay,Publisher,RatingDist5,RatingDist4,RatingDist3,RatingDist2,RatingDist1,RatingDistTotal,CountsOfReview,Language,pagesNumber,Description,Count of text reviews
0,1300002,Still River,Harry Hunsicker,0312940904,3.69,2006,27,6,Minotaur Books,5:35,4:50,3:52,2:12,1:2,total:151,20,NaN,304,"<i>In a stark and haunting debut novel, Harry ...",20
1,1300003,The Naturalist on the River Amazons,Henry Walter Bates,1589761979,3.91,2002,15,8,Narrative Press,5:23,4:25,3:13,2:5,1:2,total:68,11,eng,432,Charles Darwin encouraged Bates to write this ...,11
2,1300017,Molecular Biology of Membrane Transport Disorders,Stanley G. Schultz,0306451646,0.00,1996,30,6,Springer,5:0,4:0,3:0,2:0,1:0,total:0,0,NaN,657,When the six of us gathered to start planning ...,0
3,1300027,The Question Of Discrimination: Racial Inequal...,Steven Shulman,0819552143,0.00,1989,1,9,Wesleyan,5:0,4:0,3:0,2:0,1:0,total:0,0,NaN,394,NaN,0
4,1300028,Unlevel Playing Fields,Randy Albelda,0070009686,3.58,1997,1,1,McGraw-Hill Companies,5:8,4:7,3:3,2:3,1:3,total:24,0,NaN,256,"Brief, topical, and issues-oriented, this inno...",0


In [5]:


# df = pd.read_csv(zf.open('book1500k-1600k.csv'))
import zipfile as ZipFile
with zipfile.ZipFile('/content/gdrive/MyDrive/BookRec.zip', 'r') as zipObj:
   # Get list of files names in zip
   listOfbookfiles = zipObj.namelist()
   finaldf = pd.DataFrame()
   for elem in listOfbookfiles:           
        if elem.startswith('book'):
            df=pd.read_csv(zf.open(elem))
            print(df.shape)
# df.shape
# df.summary()


(58292, 18)
(39705, 20)
(57046, 18)
(41892, 20)
(43622, 20)
(38288, 20)
(34759, 20)
(33439, 20)
(32986, 20)
(32105, 19)
(38863, 19)
(43561, 19)
(395957, 19)
(56182, 18)
(256595, 19)
(56586, 18)
(280256, 19)
(55155, 18)
(54859, 18)
(55156, 19)
(54273, 20)
(49843, 20)
(40890, 20)


In [6]:
import zipfile as ZipFile
with zipfile.ZipFile('/content/gdrive/MyDrive/BookRec.zip', 'r') as zipObj:
   # Get list of files names in zip
   listOfiles = zipObj.namelist()
   books_df = pd.DataFrame()
   for elem in listOfiles:           
        if elem.startswith('book'):
            df=pd.read_csv(zf.open(elem))
#             print(list(df.columns))
            books_df= pd.concat([df, books_df],axis=0)
# books_df.head(10)
books_df.shape

(1850310, 21)

In [7]:
books_df.head(5)

,Id,Name,Authors,ISBN,Rating,PublishYear,PublishMonth,PublishDay,Publisher,RatingDist5,...,RatingDist3,RatingDist2,RatingDist1,RatingDistTotal,CountsOfReview,Language,pagesNumber,Description,Count of text reviews,PagesNumber
0,900000,Cabinet 03: Weather,Jeffrey Kastner,1932698019,4.00,2004,2,1,Cabinet,5:0,...,3:0,2:0,1:0,total:2,0,NaN,112.0,"Artwork by Joe Amrhein, Maria Blondeel, Spence...",0.0,NaN
1,900001,"Workouts in a Binder for Swimmers, Triathletes...",Nick Hansen,1931382743,4.48,2005,18,11,Velo Press,5:24,...,3:3,2:0,1:0,total:44,2,NaN,101.0,This waterproof book of 100 swimming workouts ...,2.0,NaN
2,900002,Las 120 jornadas de Sodoma,Marquis de Sade,8472233847,3.12,1992,1,3,TusQuets,5:1745,...,3:2935,2:1753,1:1408,total:10087,9,NaN,424.0,The 120 Days of Sodom by Marquis de Sade relat...,9.0,NaN
3,900004,The Traveler: An American Odyssey in the Himal...,Eric Hansen,0871563509,4.18,1995,6,6,"Random House, Inc.",5:6,...,3:3,2:0,1:0,total:17,1,NaN,102.0,This loving memorial to Swift is a moving phot...,1.0,NaN
4,900006,Vertical Smile,Richard Condon,0297994026,3.74,1972,20,4,Littlehampton Book Services Ltd,5:7,...,3:13,2:2,1:0,total:39,1,NaN,334.0,NaN,1.0,NaN


In [8]:
books_df.shape

(1850310, 21)

In [9]:
books_df.drop(['PagesNumber'], axis=1, inplace=True)

In [10]:
books_df['pagesNumber'].value_counts()

32.0       22348
256.0      21473
192.0      18992
224.0      18386
128.0      17051
           ...  
3026.0         1
2019.0         1
15189.0        1
4272.0         1
4242.0         1
Name: pagesNumber, Length: 2436, dtype: int64

In [11]:
books_df['pagesNumber'].isnull().sum()

1015232

In [12]:
books_df['pagesNumber'] = books_df['pagesNumber'].fillna(256)

In [13]:
# books_df['Count of text reviews'].value_counts()

In [14]:
books_df['Language'].value_counts()

eng      180338
en-US     21489
fre       16327
ger       11470
en-GB      7767
          ...  
chp           1
ssw           1
ada           1
zap           1
myn           1
Name: Language, Length: 124, dtype: int64

In [15]:
books_df['Language'].isnull().sum()

1598488

In [16]:
books_df['Language'] = books_df['Language'].fillna('eng')

In [17]:
books_df['Language'].isnull().sum()

0

In [18]:
books_df['Authors'].isnull().sum()

0

To replace all versions of English with one standard one. 

In [19]:
books_df['Language']=books_df['Language'].replace({'en-US':'eng', 'en-GB':'eng', 'en-CA':'eng'})

In [20]:
books_df['Language'].value_counts()

eng    1808160
fre      16327
ger      11470
spa       7245
jpn       2059
        ...   
myv          1
zul          1
nub          1
ada          1
myn          1
Name: Language, Length: 121, dtype: int64

In [21]:
books_df.shape

(1850310, 20)

We want to keep only English language books in our dataframe, so we'll remove those written in other languages eg. French,Japanese etc. 

In [22]:
books_df = books_df[books_df['Language'] == 'eng']
books_df.shape

(1808160, 20)

In [23]:
books_df['Language'].value_counts()

eng    1808160
Name: Language, dtype: int64

In [24]:
# books_df[books_df['Name']=='Siddhartha']

In [25]:
import zipfile as ZipFile
with zipfile.ZipFile('/content/gdrive/MyDrive/BookRec.zip', 'r') as zipObj:
   # Get list of files names in zip
   listOfiles = zipObj.namelist()
   for elem in listOfiles:
      print(elem)

book1-100k.csv
book1000k-1100k.csv
book100k-200k.csv
book1100k-1200k.csv
book1200k-1300k.csv
book1300k-1400k.csv
book1400k-1500k.csv
book1500k-1600k.csv
book1600k-1700k.csv
book1700k-1800k.csv
book1800k-1900k.csv
book1900k-2000k.csv
book2000k-3000k.csv
book200k-300k.csv
book3000k-4000k.csv
book300k-400k.csv
book4000k-5000k.csv
book400k-500k.csv
book500k-600k.csv
book600k-700k.csv
book700k-800k.csv
book800k-900k.csv
book900k-1000k.csv
user_rating_0_to_1000.csv
user_rating_1000_to_2000.csv
user_rating_2000_to_3000.csv
user_rating_3000_to_4000.csv
user_rating_4000_to_5000.csv
user_rating_5000_to_6000.csv
user_rating_6000_to_11000.csv


In [26]:
import zipfile as ZipFile
with zipfile.ZipFile('/content/gdrive/MyDrive/BookRec.zip', 'r') as zipObj:
   # Get list of files names in zip
   listOfiles = zipObj.namelist()
   finaldf = pd.DataFrame()
   for elem in listOfiles:           
        if elem.startswith('user'):
            df=pd.read_csv(zf.open(elem))
            print(df.shape)
            #finaldf= pd.concat([df, finaldf],axis=1).drop_duplicates()
#finaldf.shape
# finaldf.head(100)

(51945, 3)
(42986, 3)
(30633, 3)
(46970, 3)
(46903, 3)
(15481, 3)
(127678, 3)


In [27]:
import zipfile as ZipFile
with zipfile.ZipFile('/content/gdrive/MyDrive/BookRec.zip', 'r') as zipObj:
   # Get list of files names in zip
   listOfiles = zipObj.namelist()
   finaldf = pd.DataFrame()
   for elem in listOfiles:           
        if elem.startswith('user'):
            df=pd.read_csv(zf.open(elem))
#             print(list(df.columns))
            finaldf= pd.concat([df, finaldf],axis=0)
finaldf.shape
# finaldf.head(100)

(362596, 3)

In [28]:
finaldf.head(5)

,ID,Name,Rating
0,6675,"Baxter, the Pig Who Wanted to Be Kosher",it was amazing
1,6675,Set This House in Order,really liked it
2,6675,Paradise Park,really liked it
3,7027,Paradise Park,liked it
4,6675,The Dead Fish Museum,it was amazing


Using value counts on the ID column we can see the number of unique users in our dataset. 

In [29]:
finaldf.ID.value_counts()

4196    3637
4806    3236
4693    2736
1134    2089
284     2052
        ... 
7399       1
5971       1
7400       1
7401       1
4828       1
Name: ID, Length: 8919, dtype: int64

In [30]:
finaldf.Name.value_counts()

Rating                                    4765
The Catcher in the Rye                     985
The Great Gatsby                           885
The Da Vinci Code (Robert Langdon, #2)     846
To Kill a Mockingbird                      830
                                          ... 
فریدریش نیچه (نسل قلم 84)                    1
ژان پل سارتر (نسل قلم 61)                    1
زیگموند فروید (نسل قلم 31)                   1
دل دلدادگی، جلد اول                          1
The Encantadas and Other Stories             1
Name: Name, Length: 103533, dtype: int64

In [31]:
finaldf.Rating.value_counts()

really liked it                      132808
liked it                              96047
it was amazing                        92354
it was ok                             28811
did not like it                        7811
This user doesn't have any rating      4765
Name: Rating, dtype: int64

In [32]:
index_names = finaldf[finaldf['Name'] == 'Rating'].index
finaldf.drop(index_names, inplace = True) #drop rows with no ratings 

In [33]:
finaldf.Rating.value_counts()

really liked it    127291
liked it            92022
it was amazing      88159
it was ok           27616
did not like it      7503
Name: Rating, dtype: int64

In [34]:
finaldf.shape

(342591, 3)

In [35]:
finaldf.head(5)

,ID,Name,Rating
580,7064,Siddhartha,it was amazing
581,7077,Siddhartha,it was amazing
582,7112,Siddhartha,it was amazing
583,7126,Siddhartha,it was amazing
584,7175,Siddhartha,liked it


To remove the Users/User IDs that have rated less than 20 books, we can see that only 2163 such users remain and these are the only ones whose ratings we want to keep. 

In [36]:
ID_grp= finaldf.groupby(['ID']).size().reset_index(name='counts')
print(ID_grp.shape)
ID_grp
x = finaldf.groupby('ID').count()['Rating'] >= 20
regular_users = x[x].index
type(regular_users)


(4118, 2)


pandas.core.indexes.numeric.Int64Index

In [37]:
filtered_rating = finaldf[finaldf['ID'].isin(regular_users)]
filtered_rating.head(5)

,ID,Name,Rating
580,7064,Siddhartha,it was amazing
581,7077,Siddhartha,it was amazing
582,7112,Siddhartha,it was amazing
583,7126,Siddhartha,it was amazing
584,7175,Siddhartha,liked it


In [38]:
filtered_rating.shape

(330459, 3)

In [39]:
filtered_rating.ID.value_counts()

4196    3582
4806    3149
4693    2685
1134    2086
284     2052
        ... 
7249      20
9437      20
4501      20
4784      20
9814      20
Name: ID, Length: 2163, dtype: int64

In [40]:
pd.set_option('display.max_rows', 1000)

In [41]:
y = filtered_rating.groupby('Name').count()['Rating']>=20
frequent_books = y[y].index

In [42]:
final_ratings = filtered_rating[filtered_rating['Name'].isin(frequent_books)]


In [43]:
final_ratings.Name.value_counts()

The Catcher in the Rye                                         822
The Great Gatsby                                               762
To Kill a Mockingbird                                          724
1984                                                           626
Animal Farm                                                    611
                                                              ... 
Dear John                                                       20
مجموعه کامل اشعار                                               20
Origin (Robert Langdon, #5)                                     20
The Physick Book of Deliverance Dane (The Physick Book, #1)     20
My Sweet Orange Tree                                            20
Name: Name, Length: 2270, dtype: int64

In [44]:
final_ratings.shape

(132305, 3)

In [45]:
from sklearn.preprocessing import LabelEncoder
# le = LabelEncoder()
# Assigning numerical values and storing in another column
final_ratings['Rating_Cat'] = final_ratings['Rating']
final_ratings['Rating_Cat']= final_ratings['Rating_Cat'].replace({'did not like it': 1, 'it was ok': 2, 'liked it': 3, 'really liked it': 4, 'it was amazing': 5})
final_ratings.drop(['Rating'],axis=1,inplace=True)
final_ratings.rename({'ID': 'User_ID'}, axis=1, inplace=True)
final_ratings.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guid

,User_ID,Name,Rating_Cat
580,7064,Siddhartha,5
581,7077,Siddhartha,5
582,7112,Siddhartha,5
583,7126,Siddhartha,5
584,7175,Siddhartha,3


In [46]:
books_list=final_ratings['Name'].unique().tolist()
# print (len(books_list))
books_dict = dict.fromkeys(books_list, 0)
i=100
for k, v in books_dict.items():
    books_dict[k]=i
    i+=1
#     print(books_dict[k])  
# books_dict is the dictionary with names of books and their unique ids stored as key-value pairs 


In [47]:
# to look at five key-value pairs from the dict since the entire thing can't be viewed
key_values_5 = {k: books_dict[k] for k in list(books_dict)[:100]}
key_values_5

{'A Clash of Kings (A Song of Ice and Fire, #2)': 152,
 'A Dance with Dragons (A Song of Ice and Fire, #5)': 148,
 'A Discovery of Witches (All Souls Trilogy, #1)': 155,
 'A Feast for Crows (A Song of Ice and Fire, #4)': 149,
 'A Game of Thrones (A Song of Ice and Fire, #1)': 153,
 'A Storm of Swords (A Song of Ice and Fire, #3)': 151,
 'A Visit from the Goon Squad': 163,
 'All the Light We Cannot See': 108,
 'American Gods (American Gods, #1)': 127,
 'Americanah': 126,
 'Angels & Demons (Robert Langdon, #1)': 101,
 'Arcadia': 139,
 'Attachments': 113,
 'Beautiful Ruins': 145,
 'Bloodsucking Fiends (A Love Story, #1)': 131,
 'Bossypants': 162,
 'Breaking Dawn (Twilight, #4)': 180,
 'Catching Fire (The Hunger Games, #2)': 170,
 'Dept. of Speculation': 106,
 'Dress Your Family in Corduroy and Denim': 191,
 'Eclipse (Twilight, #3)': 184,
 'Fates and Furies': 102,
 'Fear of Flying': 146,
 'Fingersmith': 156,
 'Freedom': 168,
 'Gone Girl': 143,
 'Good Omens: The Nice and Accurate Prophecies

In [48]:
def get_book_name(val):
    for key, value in books_dict.items():
        #  print(val)
         if value == val:
             
             return key

    return ("book for corresponding ID doesn't exist")

name=get_book_name(1486)
name

'The Reptile Room (A Series of Unfortunate Events, #2)'

In [49]:
final_ratings["Book_ID"] = final_ratings["Name"].apply(lambda x: books_dict.get(x))
final_ratings.drop(['Name'], axis=1,inplace=True)
final_ratings.head(10)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,User_ID,Rating_Cat,Book_ID
580,7064,5,100
581,7077,5,100
582,7112,5,100
583,7126,5,100
584,7175,3,100
585,7206,3,100
586,7210,4,100
587,7227,2,100
588,7256,3,100
589,7267,4,100


In [50]:
final_ratings = final_ratings.reindex(['User_ID','Book_ID','Rating_Cat'], axis=1)
final_ratings.head()

,User_ID,Book_ID,Rating_Cat
580,7064,100,5
581,7077,100,5
582,7112,100,5
583,7126,100,5
584,7175,100,3


In [51]:
final_ratings.shape

(132305, 3)

In [52]:
final_ratings.Book_ID.value_counts()

380     822
524     762
339     724
370     626
529     611
       ... 
1291     20
2134     20
1258     20
1221     20
678      20
Name: Book_ID, Length: 2270, dtype: int64

In [53]:
#dividing testing and training data 
#Use lightFM to split data into train and test set 
# from sklearn.model_selection import train_test_split
# train_df, test_df = train_test_split(comb_df, test_size=0.8)

In [54]:
# train_df.shape

In [55]:
pivot_df = final_ratings.pivot_table(index='Book_ID',columns='User_ID',values='Rating_Cat',fill_value=0)
# pivot_df = final_ratings.pivot_table(values = 'Rating_Cat', index = 'Book_ID', columns = 'User_ID')  
# pivot_df.head(50)
pivot_df.shape

(2270, 2151)

In [56]:
pivot_df.fillna(0, inplace = True)


In [57]:
# from sklearn.decomposition import TruncatedSVD

# # X = train_pivot_df.T
# SVD = TruncatedSVD(n_components=50, random_state=20)
# SVD_matrix = SVD.fit_transform(train_pivot_df)


In [58]:
from sklearn.decomposition import TruncatedSVD

# # X = train_pivot_df.T
# SVD = TruncatedSVD(n_components=50, random_state=20)
# SVD_matrix = SVD.fit_transform(pt)
svd = TruncatedSVD(n_components=1000, random_state=20)
svd.fit(pivot_df)
result = svd.transform(pivot_df)
# print(result)

In [59]:
result.shape

(2270, 1000)

In [60]:
svd.explained_variance_ratio_.sum()

0.9634070587092624

In [61]:
from sklearn.metrics.pairwise import cosine_similarity
similarity_scores = cosine_similarity(result)

In [62]:
type(similarity_scores)

numpy.ndarray

In [63]:
similarity_scores[5]

array([0.09290509, 0.21751544, 0.30194623, ..., 0.08450835, 0.01545512,
       0.03497742])

In [64]:
book_sim_df = pd.DataFrame(data=similarity_scores, index= pivot_df.index, columns= pivot_df.index)
# book_sim_df


In [65]:
book_sim_df.values[[np.arange(book_sim_df.shape[0])]*2] = 0
book_sim_df

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  """Entry point for launching an IPython kernel.


Book_ID,100,101,102,103,104,105,106,107,108,109,...,2360,2361,2362,2363,2364,2365,2366,2367,2368,2369
Book_ID,,,,,,,,,,,,,,,,,,,,,
100,0.000000,0.126587,0.060658,0.011265,0.043016,0.092905,0.064397,0.070271,0.102704,0.094952,...,0.084318,0.017219,0.042005,0.087281,0.147371,0.111888,0.041116,0.048242,0.064304,0.122979
101,0.126587,0.000000,0.091728,0.118922,0.139198,0.217515,0.034795,0.075147,0.163848,0.111820,...,0.012181,0.123150,0.020797,0.097432,0.186379,0.077510,0.028251,0.029784,0.029150,0.023469
102,0.060658,0.091728,0.000000,0.202212,0.303745,0.301946,0.232300,0.065701,0.374648,0.067763,...,0.001153,0.066356,0.000350,0.178601,0.205091,0.095425,0.000194,0.168340,0.028666,0.118229
103,0.011265,0.118922,0.202212,0.000000,0.461009,0.355952,0.046730,0.040709,0.219905,0.060919,...,0.001470,0.067987,-0.001101,0.087714,0.114847,0.040672,0.010384,0.024274,-0.004391,0.039911
104,0.043016,0.139198,0.303745,0.461009,0.000000,0.413387,0.122256,0.032483,0.338222,0.061211,...,0.001184,0.144724,-0.000020,0.153259,0.183844,0.030091,0.007024,0.039057,-0.003443,0.058323
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2365,0.111888,0.077510,0.095425,0.040672,0.030091,0.136918,0.064745,0.042938,0.084768,0.052888,...,0.000539,0.011613,-0.001225,0.090369,0.303025,0.000000,0.131584,0.104953,0.034354,0.047939
2366,0.041116,0.028251,0.000194,0.010384,0.007024,0.061906,0.033791,0.022815,0.030345,0.027124,...,-0.001896,0.022449,-0.000924,0.047927,0.129243,0.131584,0.000000,0.154861,0.052511,0.024633
2367,0.048242,0.029784,0.168340,0.024274,0.039057,0.084508,0.168675,0.019382,0.107437,0.016510,...,0.001303,0.020545,-0.002617,0.159493,0.187759,0.104953,0.154861,0.000000,0.197702,0.219060


In [66]:
recommend=book_sim_df[book_sim_df[195]>0.35].index
recommend

# Recommend = list(X.index[correlation_product_ID > 0.90])
# Recommend[0:9]

Int64Index([190, 199], dtype='int64', name='Book_ID')

In [67]:
book=get_book_name(110)
book


'The Martian'

In [68]:
def list_of_recs(ID, df=book_sim_df):
  # print (df.nlargest(5, ID))
   a = df.sort_values(ID, ascending=False).head(6)
   rec_list=[]
   i=0
   for i in range(0,5):
    rec_list.append(get_book_name(a.index.values[i]))
    i+=1

   return(rec_list)
  #  print first five row indexes of this dataframe, for each of the indexes call the 
  #  get_book_name function and append the book names to a list which will be returned by this function. 


rec=list_of_recs(110)
rec

['Ready Player One (Ready Player One, #1)',
 'Artemis',
 'Seveneves',
 'Red Rising (Red Rising Saga, #1)',
 'Station Eleven']